# Actor-critic learning for the CVRP

## Importing the libraries

In [1]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from gym import wrappers
from torch.autograd import Variable
from collections import deque

## Creating a CVRP environment

In [2]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import numpy as np
import random
import copy
import math

class VRPEnv(gym.Env):
  def __init__(self):
    # customer count ('0' is depot) 
    self.customer_count = 11
    # the capacity of vehicles
    self.vehicle_capacity = 2
  
    self.action_space = spaces.Discrete(3)
    self.observation_space = spaces.Box(low=0,high=1, shape=(4,1), dtype=np.float64)
    self.VRP = np.array((self.customer_count,4))
    self._max_episode_steps = 1000
    self.viewer = None
    self.state = None
    self.steps_beyond_done = None
    self.route = []
    self.route.append(0)
    self.previous_action = 0
    self.hn_actor = torch.zeros([1,self.customer_count,128], dtype=torch.float32).to(device)
    self.hn_actor_target = torch.zeros([1,self.customer_count,128], dtype=torch.float32).to(device)
    

  def reset(self, seed=200):
    if seed == 200:
      seed = int(time.time())
    np.random.seed(seed)
    x_locations = (np.random.rand(self.customer_count)).reshape((self.customer_count,1))
    y_locations = (np.random.rand(self.customer_count)).reshape((self.customer_count,1))
    demand = (np.random.randint(1,9,self.customer_count).reshape((self.customer_count,1))).reshape((self.customer_count,1))/10 # Normalise to between 0.1 and 0.9
    capacity = np.repeat(self.vehicle_capacity,self.customer_count).reshape((self.customer_count,1))
    VRP = np.concatenate((np.concatenate((np.concatenate((x_locations,y_locations), axis=1),demand),axis=1),capacity),axis=1)
    self.VRP = VRP.reshape((self.customer_count,4))
    self.unserved_customers = []
    for i in range(1, self.customer_count):
      self.unserved_customers.append(i)
    self.routes = []
    self.route = []
    self.route.append(0)
    self.VRP[0,2] = 0 # Set the demand at thedepot to 0
    self.state = copy.deepcopy(self.VRP)
    self.previous_action = 0
    return self.state
  

  def step(self, action):
    # Calculate the reward as the negative euclidean distance
    reward = -((self.state[self.previous_action,0]-self.state[action,0])**2+(self.state[self.previous_action,1]-self.state[action,1])**2)**0.5 # - Euclidean distance between customers
    load = self.state[0,3]
    self.state[:,3] = max(0,(load-self.state[action,2])) # Update the vehicle load
    self.state[action, 2] = max(0,self.state[action,2]-load) # Update the demand at served customer
    done = False
    if action == 0: # Return to the depot
      self.route.append(action) # End route
      self.routes.append(self.route) # Add subroute to list of all routes
      self.route = [] # Initiate new subroute
      self.state[:,3] = self.vehicle_capacity # Refill the vehicle
    self.route.append(action) # Add action to the subroute
    if max(self.state[:,2]) > 0: # If there are unserved customers left
      done = False
    elif max(self.state[:,2]) == 0 and action == 0: # If there are no unserved customers left and we have returned to the depot
      done = True
      self.route.append(0)
    self.previous_action = action # Update the previous action
    return self.state, reward, done


## Let's test the environment step function

In [3]:
env = VRPEnv() # Create an instance of the environment
state = env.reset() # Reset the environment
action = 2 # Perform action with customer 2
print(state)
state, reward, done = env.step(action) # Perform the actual transition
print(state)

[[0.11437463 0.35433916 0.         2.        ]
 [0.90856365 0.30937283 0.3        2.        ]
 [0.59436509 0.75216188 0.7        2.        ]
 [0.50406788 0.0964249  0.8        2.        ]
 [0.90713434 0.04520073 0.6        2.        ]
 [0.51126682 0.32464299 0.2        2.        ]
 [0.85039854 0.7478735  0.3        2.        ]
 [0.35165363 0.10587075 0.1        2.        ]
 [0.28080822 0.34687137 0.1        2.        ]
 [0.35065381 0.80720308 0.1        2.        ]
 [0.68470168 0.90565026 0.2        2.        ]]
[[0.11437463 0.35433916 0.         1.3       ]
 [0.90856365 0.30937283 0.3        1.3       ]
 [0.59436509 0.75216188 0.         1.3       ]
 [0.50406788 0.0964249  0.8        1.3       ]
 [0.90713434 0.04520073 0.6        1.3       ]
 [0.51126682 0.32464299 0.2        1.3       ]
 [0.85039854 0.7478735  0.3        1.3       ]
 [0.35165363 0.10587075 0.1        1.3       ]
 [0.28080822 0.34687137 0.1        1.3       ]
 [0.35065381 0.80720308 0.1        1.3       ]
 [0.68470168

## Initialize the Experience Replay memory

In [4]:
class ReplayBuffer(object):

  def __init__(self, max_size=1e6):
    self.storage = []
    self.max_size = max_size
    self.ptr = 0

  def add(self, transition):
    if len(self.storage) == self.max_size:
      #self.storage[int(self.ptr)] = transition
      #self.ptr = (self.ptr + 1) % self.max_size
      self.storage.pop(0)
      self.storage.append(transition)
    else:
      self.storage.append(transition)

  def sample(self, batch_size):
    ind =  np.arange((len(self.storage)-(batch_size+1)),len(self.storage)-1,1) #np.random.randint(0, len(self.storage), size=batch_size)
    batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], []
    for i in ind: 
      state, next_state, action, reward, done = self.storage[i]
      batch_states.append(np.array(state, copy=False))
      batch_next_states.append(np.array(next_state, copy=False))
      batch_actions.append(np.array(action, copy=False))
      batch_rewards.append(np.array(reward, copy=False))
      batch_dones.append(np.array(done, copy=False))
    return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), np.array(batch_dones).reshape(-1, 1)

## Build the neural network for the Actor and Actor-target models - contains the attention mechanism

In [5]:
class Actor(nn.Module):
  
  def __init__(self, state_dim=4, embed_size = 128):#, action_dim, max_action):
    super(Actor, self).__init__()
    self.embed = nn.Linear((state_dim), embed_size) # Encoding to higher dimensional space - can also be changed to convolutional layer as in the paper
    self.u_t = nn.RNN(embed_size,embed_size,1) # RNN Layer for the attention mechanism
    self.v_t_a = nn.Linear(embed_size,1) # Linear for getting u_t
    self.bar_u_t = nn.RNN(embed_size,embed_size,1) # RNN Layer for the context vector
    self.a_t = nn.Softmax(dim = 1) # Softmax layer for the attention mechanism
    self.v_t_u = nn.Linear(embed_size,1) # Linear for getting u_t
    self.final = nn.Softmax(dim = 1) # Softmax layer for the final output

  def forward(self, x, hn = env.hn_actor):
    cond1 = (x[:,:,2]<x[:,:,3]).int() # Can we meet the demand
    cond2 = (x[:,:,2]>0).int() # Is there demand at the customer
    mask1 = torch.minimum(cond1,cond2) # Select those customers with demand, and whose demand we can meet
    mask1 = torch.reshape(mask1,(len(x),env.customer_count,1))
    x = self.embed(x)
    u, hn = self.u_t(x, hn)
    u = self.v_t_a(u)
    a = self.a_t(u) # Up to equation (4) now
    c = torch.randn(x.shape)
    c = torch.mul(x,a)
    c = torch.sum(c, 0)
    c = torch.reshape(c,(1,env.customer_count,128))
    u_bar, hu = self.bar_u_t(x,c)
    u_bar = self.v_t_u(u_bar)
    output = self.final(u_bar)
    output = torch.mul(output,mask1)
    return output

class Actor_Target(nn.Module):
  
  def __init__(self, state_dim=4, embed_size = 128):#, action_dim, max_action):
    super(Actor_Target, self).__init__()
    self.embed = nn.Linear((state_dim), embed_size) # Encoding to higher dimensional space - can also be changed to convolutional layer as in the paper
    self.u_t = nn.RNN(embed_size,embed_size,1) # RNN Layer for the attention mechanism
    self.v_t_a = nn.Linear(embed_size,1) # Linear for getting u_t
    self.bar_u_t = nn.RNN(embed_size,embed_size,1) # RNN Layer for the context vector
    self.a_t = nn.Softmax(dim = 1) # Softmax layer for the attention mechanism
    self.v_t_u = nn.Linear(embed_size,1) # Linear for getting u_t
    self.final = nn.Softmax(dim = 1) # Softmax layer for the final output

  def forward(self, x, hn = env.hn_actor_target):
    cond1 = (x[:,:,2]<x[:,:,3]).int() # Can we meet the demand
    cond2 = (x[:,:,2]>0).int() # Is there demand at the customer
    mask1 = torch.minimum(cond1,cond2) # Select those customers with demand, and whose demand we can meet
    mask1 = torch.reshape(mask1,(len(x),env.customer_count,1))
    x = self.embed(x)
    u, hn = self.u_t(x, hn)
    u = self.v_t_a(u)
    a = self.a_t(u) # Up to equation (4) now
    c = torch.randn(x.shape)
    c = torch.mul(x,a)
    c = torch.sum(c, 0)
    c = torch.reshape(c,(1,env.customer_count,128))
    u_bar, hu = self.bar_u_t(x,c)
    u_bar = self.v_t_u(u_bar)
    output = self.final(u_bar)
    output = torch.mul(output,mask1)
    return output

## Build the neural network for the Critic and Critic-target model

In [6]:
class Critic(nn.Module):
  
  def __init__(self, state_dim=4, action_dim = env.customer_count, embed_size = 128):
    super(Critic, self).__init__()
    # Defining the first Critic neural network
    self.layer_1 = nn.Linear(state_dim, embed_size) # Perform the embedding
    self.layer_2 = nn.Linear(embed_size, embed_size) # Adding the single dense layer
    self.layer_3 = nn.Linear(embed_size, 1) # Adding the output layer


  def forward(self, x, u): # x is the state, u is the action
    # Forward-Propagation on the Critic Neural Network
    x1 = F.relu(self.layer_1(x))
    ws = torch.mul(x1,u)
    x2 = F.relu(self.layer_2(ws))
    x2 = self.layer_3(x2)
    x2 = torch.sum(x2,1)
    return x2


## Testing the actor and critic networks to confirm their output

In [7]:
env = VRPEnv()
env.reset()
state = env.reset()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
state = torch.Tensor(state.reshape(1,env.customer_count,4)).to(device)
actor = Actor().to(device)
actor_target = Actor_Target().to(device)
prediction = actor(state)
prediction_target = actor_target(state)
print(prediction)
critic = Critic().to(device)
q_value = critic(state,prediction)
print(q_value)

tensor([[[0.0000],
         [0.0941],
         [0.0902],
         [0.1007],
         [0.1012],
         [0.0910],
         [0.0877],
         [0.0926],
         [0.0887],
         [0.0828],
         [0.0840]]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([[-0.4111]], device='cuda:0', grad_fn=<SumBackward1>)


## Training Process

In [8]:
# Selecting the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.autograd.set_detect_anomaly(True)

# Building the Training Process into a class
class Actor_Critic(object):
  
  def __init__(self, state_dim):
    self.actor = Actor(state_dim).to(device)
    self.actor_target = Actor_Target(state_dim).to(device)
    self.actor_target.load_state_dict(self.actor.state_dict())
    self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr = 0.0001)
    self.critic = Critic(state_dim, action_dim).to(device)
    self.critic_target = Critic(state_dim, action_dim).to(device)
    self.critic_target.load_state_dict(self.critic.state_dict())
    self.critic_optimizer = torch.optim.Adam(self.critic.parameters())
    self.max_action = max_action

  def select_action(self, state, state_dim):
    state_tensor = torch.Tensor(state.reshape(1,env.customer_count,state_dim)).to(device)
    current_Q = actor(state_tensor).detach()
    current_Q = current_Q.detach().cpu().numpy().reshape(env.customer_count)
    action = np.argmax(current_Q)
    return action, current_Q

  def select_target_action(self, state):
    state_tensor = torch.Tensor(state.reshape(1,env.customer_count,state_dim)).to(device)
    target_Q = actor_target(state_tensor).detach()
    target_Q = target_Q.detach().cpu().numpy().reshape(env.customer_count)
    action = torch.argmax(target_Q)
    return action, target_Q

  def train(self, replay_buffer, iterations, batch_size=32, discount=0.99, tau=0.005):
    
    for it in range(iterations):
      
      # Sample a batch of transitions (s, s’, a, r) from the memory
      batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)
      state = torch.Tensor(batch_states).to(device)
      next_state = torch.Tensor(batch_next_states).to(device)
      action = torch.Tensor(batch_actions).to(device)
      reward = torch.Tensor(batch_rewards).to(device)
      done = torch.Tensor(batch_dones).to(device)

      # Determine the next action based on the actor target
      next_action = self.actor_target(next_state)
      
      # Determine the target Q-value based on the critic target
      target_Q = self.critic_target(next_state, next_action)

      # Get the final target using the RL update rule
      target_Q = reward + ((1 - done) * discount * target_Q).detach()

      # Determine the probabilities of the actions chosen by the actor
      action = self.actor(state)

      # Determine the current Q-value based on the critic
      current_Q = self.critic(state,action)

      # Compute the loss coming from the critic models: Critic Loss = MSE_Loss(Q, Qt)
      critic_loss = F.mse_loss(current_Q, target_Q)

      # Backpropagate this Critic loss and update the parameters of the Critic model with Adam optimizer
      self.critic_optimizer.zero_grad()
      critic_loss.backward(retain_graph=True)
      self.critic_optimizer.step()

      # Update our Actor model by performing gradient descent on the output of the Critic model  
      actor_loss = -self.critic(state, action).mean()
      self.actor_optimizer.zero_grad()
      actor_loss.backward()
      self.actor_optimizer.step()

      # Update the weights of the Actor target by polyak averaging
      for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
        target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

      # Update the weights of the Critic target by polyak averaging
      for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
        target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
  
  # Make a save method to save a trained model
  def save(self, filename, directory):
    torch.save(self.actor.state_dict(), '%s/%s_actor.pth' % (directory, filename))
    torch.save(self.critic.state_dict(), '%s/%s_critic.pth' % (directory, filename))
  
  # Making a load method to load a pre-trained model
  def load(self, filename, directory):
    self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, filename)))
    self.critic.load_state_dict(torch.load('%s/%s_critic.pth' % (directory, filename)))


## Create a function that evaluates the policy by calculating its average reward over 10 episodes

In [9]:
def evaluate_policy(policy, eval_episodes=10):
  avg_reward = 0.
  for _ in range(eval_episodes):
    obs = env.reset()
    done = False
    while not done:
      action, current_Q = policy.select_action(obs, state_dim)
      obs, reward, done = env.step(action)
      avg_reward += reward
  avg_reward /= eval_episodes
  print ("---------------------------------------")
  print ("Average Reward over the Evaluation Step: %f" % (avg_reward))
  print ("---------------------------------------")
  return avg_reward

## Set the parameters

In [10]:
env_name = "CVRP" # Name of a environment (set it to any Continous environment you want)
seed = 0 # Random seed number
start_timesteps = 1e4 # Number of iterations/timesteps before which the model randomly chooses an action, and after which it starts to use the policy network
eval_freq = 5e3 # How often the evaluation step is performed (after how many timesteps)
max_timesteps = 5e5 # Total number of iterations/timesteps
save_models = True # Boolean checker whether or not to save the pre-trained model
batch_size = 128 # Size of the batch
discount = 0.99 # Discount factor gamma, used in the calculation of the total discounted reward
tau = 0.001 # Target network update rate

## Create a file name for the two saved models: the Actor and Critic models

In [11]:
file_name = "%s_%s_%s" % ("Actor_Critic", env_name, str(seed))
print ("---------------------------------------")
print ("Settings: %s" % (file_name))
print ("---------------------------------------")

---------------------------------------
Settings: Actor_Critic_CVRP_0
---------------------------------------


## Create a folder to save the trained models

In [12]:
if not os.path.exists("./results"):
  os.makedirs("./results")
if save_models and not os.path.exists("./pytorch_models"):
  os.makedirs("./pytorch_models")

## Create an instance of the CVRP environment

In [13]:
env = VRPEnv()

## Set seeds and get the necessary information on the states and actions in the chosen environment

In [14]:
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
state_dim = env.observation_space.shape[0]
action_dim = env.customer_count
max_action = 1

## Create the policy network

In [15]:
print(state_dim, action_dim, max_action)
policy = Actor_Critic(state_dim)

4 11 1


## Create the Experience Replay memory

In [16]:
replay_buffer = ReplayBuffer()

## Define a list where all the evaluation results over 10 episodes are stored

In [17]:
evaluations = [evaluate_policy(policy, eval_episodes=1)]
print(env.routes)

---------------------------------------
Average Reward over the Evaluation Step: -5.339425
---------------------------------------
[[0, 6, 9, 1, 10, 4, 7, 0], [0, 5, 2, 8, 0], [0, 3, 0]]


In [18]:
print(env.routes)
print(env.state)

[[0, 6, 9, 1, 10, 4, 7, 0], [0, 5, 2, 8, 0], [0, 3, 0]]
[[0.89425677 0.62202079 0.         2.        ]
 [0.31273994 0.02823563 0.         2.        ]
 [0.56317247 0.71834622 0.         2.        ]
 [0.15559268 0.93019907 0.         2.        ]
 [0.30514436 0.31988291 0.         2.        ]
 [0.15515951 0.74133423 0.         2.        ]
 [0.57963753 0.01344141 0.         2.        ]
 [0.88074163 0.72017397 0.         2.        ]
 [0.51136044 0.80973621 0.         2.        ]
 [0.33875451 0.02505394 0.         2.        ]
 [0.33504032 0.21694058 0.         2.        ]]


## Create a folder directory in which the final results will be saved

In [19]:
def mkdir(base, name):
    path = os.path.join(base, name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path
work_dir = mkdir('exp', 'brs')
monitor_dir = mkdir(work_dir, 'monitor')
max_episode_steps = env._max_episode_steps
save_env_vid = False
if save_env_vid:
  env = wrappers.Monitor(env, monitor_dir, force = True)
  env.reset()

## Initialize the training process variables

In [20]:
total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True
t0 = time.time()

## Training

In [ ]:
# We start the main loop over 500,000 timesteps
env.reset()
total_timesteps = 0
obs = copy.deepcopy(env.reset())
maximum = -1000
while total_timesteps < max_timesteps:
  
  # If the episode is done
  if done:
    
    # If we are not at the very beginning, we start the training process of the model
    if total_timesteps != 0 and total_timesteps > batch_size:
      print("Total Timesteps: {} Episode Num: {} Reward: {} Epsilon: {}".format(total_timesteps, episode_num, episode_reward, epsilon))
      policy.train(replay_buffer, episode_timesteps, batch_size, discount, tau)

    # We evaluate the episode and we save the policy
    if timesteps_since_eval >= eval_freq:
      timesteps_since_eval %= eval_freq
      print("Total timesteps: {} Epsilon: {}".format(total_timesteps, min(1,epsilon)))
      evaluations.append(evaluate_policy(policy))
      if evaluations[len(evaluations)-1] > maximum:
        policy.save(file_name, directory="./pytorch_models")
        maximum = evaluations[len(evaluations)-1]
      np.save("./results/%s" % (file_name), evaluations)
    
    # When the training step is done, we reset the state of the environment
    obs = copy.deepcopy(env.reset())
    
    # Set the Done to False
    done = False
    
    # Set rewards and episode timesteps to zero
    episode_reward = 0
    episode_timesteps = 0
    episode_num += 1
  
  # Work with epsilon-greedy
  epsilon = 50000/(total_timesteps+1)
  np.random.seed(total_timesteps)
  if np.random.rand() < min(1,epsilon):
    feasible = np.array([0])
    for i in range(1,env.customer_count):
      if obs[i,2] < obs[i,3] and obs[i,2] != 0:
        feasible = np.concatenate((feasible,np.array([i])))
    if len(feasible) > 1:
      feasible = np.delete(feasible,0)
    action = np.random.choice(feasible)
  else: # Choose greedy action
    action, current_Q = policy.select_action(obs, state_dim)
  
  # The agent performs the action in the environment, then reaches the next state and receives the reward
  new_obs, reward, done = env.step(action)

  # Check if the episode is done
  done_bool = 0 if episode_timesteps + 1 == env._max_episode_steps else float(done)

  # Increase the total reward
  episode_reward += reward

  # Store the new transition into the Experience Replay memory (ReplayBuffer)
  replay_buffer.add((obs, copy.deepcopy(new_obs), action, reward, done_bool)) # Have to use deepcopy to prevent overwriting of historic next states

  # Update the state, the episode timestep, the total timesteps, and the timesteps since the evaluation of the policy
  obs = copy.deepcopy(new_obs)
  episode_timesteps += 1
  total_timesteps += 1
  timesteps_since_eval += 1

# Add the last policy evaluation to our list of evaluations and we save our model
evaluations.append(evaluate_policy(policy))
if save_models: policy.save("%s" % (file_name), directory="./pytorch_models")
np.save("./results/%s" % (file_name), evaluations)

Total Timesteps: 134 Episode Num: 11 Reward: -7.7166221027212 Epsilon: 373.13432835820896
Total Timesteps: 147 Episode Num: 12 Reward: -7.0453468193966575 Epsilon: 340.13605442176873
Total Timesteps: 159 Episode Num: 13 Reward: -6.9251317529338445 Epsilon: 314.4654088050315
Total Timesteps: 172 Episode Num: 14 Reward: -6.57280741089108 Epsilon: 290.69767441860466
Total Timesteps: 184 Episode Num: 15 Reward: -6.555108225007234 Epsilon: 271.7391304347826
Total Timesteps: 197 Episode Num: 16 Reward: -7.047276208222322 Epsilon: 253.80710659898477
Total Timesteps: 210 Episode Num: 17 Reward: -6.895482958275187 Epsilon: 238.0952380952381
Total Timesteps: 223 Episode Num: 18 Reward: -5.889824317403686 Epsilon: 224.2152466367713
Total Timesteps: 236 Episode Num: 19 Reward: -6.677225406454144 Epsilon: 211.864406779661
Total Timesteps: 249 Episode Num: 20 Reward: -7.183859911763287 Epsilon: 200.80321285140562
Total Timesteps: 263 Episode Num: 21 Reward: -6.627466414119959 Epsilon: 190.1140684410

Total Timesteps: 1292 Episode Num: 101 Reward: -7.375219285719137 Epsilon: 38.69969040247678
Total Timesteps: 1304 Episode Num: 102 Reward: -4.714605910170475 Epsilon: 38.34355828220859
Total Timesteps: 1317 Episode Num: 103 Reward: -6.848132720129744 Epsilon: 37.96507213363705
Total Timesteps: 1329 Episode Num: 104 Reward: -6.543914129808644 Epsilon: 37.62227238525207
Total Timesteps: 1342 Episode Num: 105 Reward: -7.122810609516416 Epsilon: 37.257824143070046
Total Timesteps: 1355 Episode Num: 106 Reward: -6.409329938507055 Epsilon: 36.90036900369004
Total Timesteps: 1368 Episode Num: 107 Reward: -5.638941662466966 Epsilon: 36.54970760233918
Total Timesteps: 1380 Episode Num: 108 Reward: -7.5297128916094085 Epsilon: 36.231884057971016
Total Timesteps: 1393 Episode Num: 109 Reward: -5.503562781436424 Epsilon: 35.89375448671931
Total Timesteps: 1406 Episode Num: 110 Reward: -7.352307218590518 Epsilon: 35.56187766714083
Total Timesteps: 1419 Episode Num: 111 Reward: -4.963487535888658 E

Total Timesteps: 2426 Episode Num: 189 Reward: -6.797776877397653 Epsilon: 20.610057708161584
Total Timesteps: 2439 Episode Num: 190 Reward: -8.53843176726857 Epsilon: 20.50020500205002
Total Timesteps: 2453 Episode Num: 191 Reward: -7.9576250536062405 Epsilon: 20.383204239706483
Total Timesteps: 2466 Episode Num: 192 Reward: -9.588808003237716 Epsilon: 20.275750202757504
Total Timesteps: 2479 Episode Num: 193 Reward: -7.29000876005507 Epsilon: 20.169423154497782
Total Timesteps: 2492 Episode Num: 194 Reward: -7.128708665938453 Epsilon: 20.064205457463885
Total Timesteps: 2505 Episode Num: 195 Reward: -6.907882711637171 Epsilon: 19.960079840319363
Total Timesteps: 2518 Episode Num: 196 Reward: -7.314185601454649 Epsilon: 19.857029388403493
Total Timesteps: 2531 Episode Num: 197 Reward: -6.767081224378841 Epsilon: 19.755037534571315
Total Timesteps: 2544 Episode Num: 198 Reward: -6.1324451375658535 Epsilon: 19.654088050314467
Total Timesteps: 2557 Episode Num: 199 Reward: -5.16141901424

Total Timesteps: 3564 Episode Num: 277 Reward: -7.174044320782159 Epsilon: 14.029180695847362
Total Timesteps: 3577 Episode Num: 278 Reward: -8.024884272757578 Epsilon: 13.97819401733296
Total Timesteps: 3590 Episode Num: 279 Reward: -6.652580954866882 Epsilon: 13.927576601671309
Total Timesteps: 3603 Episode Num: 280 Reward: -5.797428674827902 Epsilon: 13.877324451845684
Total Timesteps: 3616 Episode Num: 281 Reward: -6.914752280910663 Epsilon: 13.827433628318584
Total Timesteps: 3629 Episode Num: 282 Reward: -6.71876858104695 Epsilon: 13.777900248002204
Total Timesteps: 3643 Episode Num: 283 Reward: -8.41910557728666 Epsilon: 13.724951962668131
Total Timesteps: 3656 Episode Num: 284 Reward: -5.980656932648756 Epsilon: 13.676148796498905
Total Timesteps: 3669 Episode Num: 285 Reward: -7.221372485644578 Epsilon: 13.627691469065141
Total Timesteps: 3682 Episode Num: 286 Reward: -6.340578494974445 Epsilon: 13.579576317218903
Total Timesteps: 3695 Episode Num: 287 Reward: -5.4934579772830

Total Timesteps: 4705 Episode Num: 365 Reward: -6.171720214407562 Epsilon: 10.626992561105208
Total Timesteps: 4719 Episode Num: 366 Reward: -6.675785042837068 Epsilon: 10.595465140919686
Total Timesteps: 4732 Episode Num: 367 Reward: -5.557286485297405 Epsilon: 10.566356720202874
Total Timesteps: 4746 Episode Num: 368 Reward: -6.271716537999604 Epsilon: 10.53518752633797
Total Timesteps: 4759 Episode Num: 369 Reward: -5.792138446853599 Epsilon: 10.506408909434755
Total Timesteps: 4772 Episode Num: 370 Reward: -7.045041591105368 Epsilon: 10.477787091366304
Total Timesteps: 4785 Episode Num: 371 Reward: -7.839936730691921 Epsilon: 10.449320794148381
Total Timesteps: 4798 Episode Num: 372 Reward: -6.707652032969159 Epsilon: 10.421008753647353
Total Timesteps: 4811 Episode Num: 373 Reward: -8.205094222144142 Epsilon: 10.392849719393057
Total Timesteps: 4824 Episode Num: 374 Reward: -6.728769143084986 Epsilon: 10.364842454394694
Total Timesteps: 4838 Episode Num: 375 Reward: -7.20250330163

...

Total Timesteps: 148462 Episode Num: 11531 Reward: -7.883490474592103 Epsilon: 0.33678651776212093
Total Timesteps: 148476 Episode Num: 11532 Reward: -6.880908181643764 Epsilon: 0.3367547617123306
Total Timesteps: 148490 Episode Num: 11533 Reward: -6.29303869619942 Epsilon: 0.3367230116506162
Total Timesteps: 148502 Episode Num: 11534 Reward: -4.941365679135667 Epsilon: 0.3366958020767397
Total Timesteps: 148515 Episode Num: 11535 Reward: -5.840921883317822 Epsilon: 0.3366663300003367
Total Timesteps: 148527 Episode Num: 11536 Reward: -6.526970370607385 Epsilon: 0.33663912958586656
Total Timesteps: 148540 Episode Num: 11537 Reward: -6.604602871148013 Epsilon: 0.3366096674296486
Total Timesteps: 148553 Episode Num: 11538 Reward: -5.431138691237709 Epsilon: 0.33658021042994757
Total Timesteps: 148566 Episode Num: 11539 Reward: -4.604654527041424 Epsilon: 0.33655075858540984
Total Timesteps: 148579 Episode Num: 11540 Reward: -5.788229224570577 Epsilon: 0.3365213118946823
Total Timesteps: 

Total Timesteps: 149547 Episode Num: 11615 Reward: -5.2905647295655935 Epsilon: 0.3343430493423472
Total Timesteps: 149560 Episode Num: 11616 Reward: -7.221984755259781 Epsilon: 0.33431398769724524
Total Timesteps: 149573 Episode Num: 11617 Reward: -4.749765019269414 Epsilon: 0.3342849311038757
Total Timesteps: 149586 Episode Num: 11618 Reward: -6.9402067248699275 Epsilon: 0.33425587956092145
Total Timesteps: 149599 Episode Num: 11619 Reward: -5.521210580386098 Epsilon: 0.33422683306706596
Total Timesteps: 149611 Episode Num: 11620 Reward: -5.345189433808126 Epsilon: 0.3342000253992019
Total Timesteps: 149624 Episode Num: 11621 Reward: -5.606979885892105 Epsilon: 0.3341709886114527
Total Timesteps: 149637 Episode Num: 11622 Reward: -6.407178718600665 Epsilon: 0.3341419568689562
Total Timesteps: 149650 Episode Num: 11623 Reward: -6.976647323592957 Epsilon: 0.3341129301703976
Total Timesteps: 149663 Episode Num: 11624 Reward: -6.993823023062259 Epsilon: 0.3340839085144625
Total Timesteps

Total Timesteps: 150600 Episode Num: 11697 Reward: -6.862482707989875 Epsilon: 0.33200531208499334
Total Timesteps: 150613 Episode Num: 11698 Reward: -4.526975798660205 Epsilon: 0.33197665540159216
Total Timesteps: 150626 Episode Num: 11699 Reward: -5.407855517466312 Epsilon: 0.33194800366470595
Total Timesteps: 150639 Episode Num: 11700 Reward: -5.564642092549983 Epsilon: 0.3319193568730541
Total Timesteps: 150652 Episode Num: 11701 Reward: -6.952579662370655 Epsilon: 0.33189071502535644
Total Timesteps: 150665 Episode Num: 11702 Reward: -4.777241459299305 Epsilon: 0.3318620781203332
Total Timesteps: 150677 Episode Num: 11703 Reward: -5.525617236369897 Epsilon: 0.3318356484400406
Total Timesteps: 150690 Episode Num: 11704 Reward: -7.105210387346189 Epsilon: 0.3318070210365651
Total Timesteps: 150703 Episode Num: 11705 Reward: -6.345660331056214 Epsilon: 0.3317783985720258
Total Timesteps: 150716 Episode Num: 11706 Reward: -6.904712885855293 Epsilon: 0.3317497810451445
Total Timesteps:

Total Timesteps: 151685 Episode Num: 11781 Reward: -5.236118216110887 Epsilon: 0.32963048422718133
Total Timesteps: 151698 Episode Num: 11782 Reward: -7.417334036143537 Epsilon: 0.3296022360215692
Total Timesteps: 151710 Episode Num: 11783 Reward: -5.562037155115277 Epsilon: 0.3295761650517435
Total Timesteps: 151723 Episode Num: 11784 Reward: -7.348672856753594 Epsilon: 0.3295479261549007
Total Timesteps: 151736 Episode Num: 11785 Reward: -4.519434836783642 Epsilon: 0.3295196920967997
Total Timesteps: 151749 Episode Num: 11786 Reward: -7.731979401797352 Epsilon: 0.32949146287619685
Total Timesteps: 151762 Episode Num: 11787 Reward: -6.504484302301267 Epsilon: 0.32946323849184905
Total Timesteps: 151775 Episode Num: 11788 Reward: -5.307440797621149 Epsilon: 0.3294350189425136
Total Timesteps: 151789 Episode Num: 11789 Reward: -5.437333862159148 Epsilon: 0.329404634064392
Total Timesteps: 151801 Episode Num: 11790 Reward: -4.153818112779834 Epsilon: 0.32937859434391076
Total Timesteps: 

Total Timesteps: 152765 Episode Num: 11865 Reward: -5.585878987526235 Epsilon: 0.32730010146303146
Total Timesteps: 152779 Episode Num: 11866 Reward: -7.4003083842758475 Epsilon: 0.3272701091118544
Total Timesteps: 152792 Episode Num: 11867 Reward: -6.7424512991153795 Epsilon: 0.3272422639928792
Total Timesteps: 152805 Episode Num: 11868 Reward: -5.620793535092468 Epsilon: 0.3272144236117928
Total Timesteps: 152818 Episode Num: 11869 Reward: -7.273250316683903 Epsilon: 0.32718658796738603
Total Timesteps: 152831 Episode Num: 11870 Reward: -6.775242305611878 Epsilon: 0.32715875705845016
Total Timesteps: 152844 Episode Num: 11871 Reward: -7.868825672094972 Epsilon: 0.3271309308837769
Total Timesteps: 152857 Episode Num: 11872 Reward: -7.008716921189482 Epsilon: 0.32710310944215837
Total Timesteps: 152869 Episode Num: 11873 Reward: -5.85914054158066 Epsilon: 0.32707743231132536
Total Timesteps: 152882 Episode Num: 11874 Reward: -4.842301077457784 Epsilon: 0.3270496199683416
Total Timestep

## The inference policy function

In [ ]:
def evaluate_final_policy(policy,random = 1, eval_episodes=1):
  avg_reward = 0.
  distance = 0.
  for k in range(eval_episodes):
    obs = env.reset(seed = random)
    done = False
    while not done:
      action, current_Q = policy.select_action(obs, state_dim)
      obs, reward, done = env.step(action)
      avg_reward += reward
    total_distance = 0
    for j in range(len(env.routes)):
      for i in range(len(env.routes[j])-1):
        total_distance = total_distance + math.floor(((env.VRP[env.routes[j][i],0]-env.VRP[env.routes[j][i+1],0])**2+(env.VRP[env.routes[j][i],1]-env.VRP[env.routes[j][i+1],1])**2)**0.5)
    distance += total_distance
  distance /= eval_episodes
  print ("---------------------------------------")
  print ("Average Distance over the Evaluation Step: %f" % (distance))
  print ("---------------------------------------")
  return distance

## Let's see if we can test the implementation

In [ ]:
results = np.zeros(1)
policy.load(file_name, './pytorch_models/')
for i in range(1):
  evaluations = [evaluate_final_policy(policy, random = i)]
  results[i] = evaluations[0]
print(results)